# Phase field modelling

In [ ]:
import numpy as np
import matplotlib.cm as cm
from matplotlib.colors import hsv_to_rgb

import matplotlib.pyplot as plt
import matplotlib.image as img
% matplotlib inline

### <span style="color: red"> Maths warning!:</span>
We have decided to introduce you to the phase field technique because it is of growing importance in materials science research, because it applies some of the important theory that you have learned (e.g. thermodynamics) and because it can be used to study many of the phenomena that you have met in your course (e.g. solidification, coarsening, etc.). We have also taken the decision to show you the mathematics that underpins the technique. We might alternatively simply have asked you to play with a phase-field simulation package, but that is a dangerous approach: it creates uninformed users and it is very easy to get *plausible rubbish* from modelling packages. Just because the results look about right, it doesn't mean they are and even when they are right, they can be right for the wrong reason. The more you know and understand about the techniques you use, the less likely you are to fall into the trap of using a black-box method to generate nonsense.

Unfortunately, the mathematics behind the phase-field approach is a little threatening. I don't expect all of you to follow all of it, and it may be that almost all of you get lost somewhere along the way, but it is important that you get a sense of what lies behind the method. We will also be using this notebook to explore practical aspects of the method and to untangle what is going on in some of the equations. So even if the mathematical notation loses you completely, you should still be able to gain some understanding of the nature of the phase field technique.

## Introduction and theory recap
You will recall from the lecture that phase field models are used to simulate a variety of features of materials at the microstructural scale. In this approach the state of the system is represented by a set of *phase variables*. Each of these variables is a mathematical (scalar) field in the sense that it has a value defined at every point in real space for our material. These phase variables might represent conserved quantities, such as the concentration of a solute, or non-conserved quantities, such as the fraction of a particular phase. Today, we will concentrate on the case of non-conserved quantities, because the equations are a little bit simpler.

### The phase variables
We will define the state of our system via a set of phase variables $\eta_i$, where $i$ runs from 1 to $p$ for a system with $p$ phases. Each of these phase variables defines the concentration of a particular phase at each point in space, so we write them as functions $\eta_i(r)$, where $r$ is a vector to a point in space. For example, in the case $p=2$ we might have the situation where at a given point in space, denoted $r^{\ast}$, the variable $\eta_1(r^{\ast})=1$ and $\eta_2(r^{\ast})=0$. We conclude that at the point $r^{\ast}$ our material has phase 1. Regions in space where the phase variables change from 0 to 1 or vice-versa therefore represent phase boundaries.

We can illustrate this with an example in one dimension. The plot below shows possible values of two phase variables as a function of $x$:

In [ ]:
# Don't worry aout the mathematical details of the functions below. 
# I am simply using a convenient form to represent a typical set of phase variables.
sigma = 1.0  # Controls interface width
r=np.linspace(-10,10,100)
plt.plot(r,1.0/(1.0+np.exp(r/sigma)),'r-', label="$\eta_1$")
plt.plot(r,1.0/(1.0+np.exp(-r/sigma)),'b-', label="$\eta_2$")
plt.legend()
plt.xlabel('Position, $r$')
plt.ylabel('Phase variable')

These phase variables represent a system that is in phase 1 on the left ($r<0$) and phase 2 on the right. There is therefore an interface between the two phases around $r=0$. Note that this interface is defined only *implicitly* by the values of $\eta_1(r)$ and $\eta_2(r)$: we do not explicitly define the interface as a feature in our model. Notice also that this interface has a finite width: it is diffuse rather than sharp. For this reason, the phase field technique is sometimes also called the *diffuse interface approach*.

#### <span style="color: red"> Task 1:</span> Explore the idea of a diffuse interface
Try replotting the above graph using different values of `sigma`. Notice how the interface always occurs at $x=0.0$, but that lower values of `sigma` give sharper interfaces and higher values give more diffuse interfaces.

### The free energy of the system
The phase field approach works on the thermodynamic principle that the state of a system will evolve towards a minimum in free energy. The next key step, then, is to define the free energy, $F$, of the system. For a system with non-conserved phase variables we write this as (don't panic, we'll work through this piece-by-piece!):

$$
F = \int \mathrm{d}^3r \left[
f_0(\eta_1(r),\eta_2(r),\dots,\eta_p(r))
+ \sum_i \frac{\kappa}{2}(\nabla \eta_i(r))^2
\right].
$$

This form for $F$ has two components. The first, $f_0$ is the bulk free energy. How about the second term? It involves the gradient of the phase variables $\eta_i$ and so will only be non-zero in regions where the values of these variables are changing. Refer back up to the plot above and you will see that this means the interface regions. The second term is thus an interfacial free energy contribution and its relative size is controlled by the size of the constant $\kappa$.

We now need a suitable form for the bulk free energy $f_0$. A common choice is a simple double-well potential:

$$
f_0(r) = 
-\frac{\alpha}{2} \sum_{i=1}^p \left(\eta_i(r)\right)^2
+\frac{\beta}{4} \sum_{i=1}^p \left(\eta_i(r)\right)^4
+\gamma\sum_{i=1}^p\sum_{j\neq i,j=1}^p \left(\eta_i(r)\right)^2\left(\eta_j(r)\right)^2
$$


### The case of a single phase

This free energy function looks a little bit threatening, so let's explore it bit by bit. First let's think about a system which can exist in only a single phase (this is not a very interesting system, of course). Consider the first two terms for a single phase variable so that the free energy takes the form:

$$
f_0(r) = 
-\frac{\alpha}{2} \left(\eta_1(r)\right)^2
+\frac{\beta}{4} \left(\eta_1(r)\right)^4.
$$

We can plot this as follows:

In [ ]:
eta=np.linspace(-1.5,1.5,100)
alpha = 1.0
beta = 1.0
plt.plot(eta,-alpha/2*eta**2+beta/4*eta**4,'r-')
plt.xlabel('$\eta_1$')
plt.ylabel('$f_0(\eta_1)$')

The first two terms define a double-well with minima at the values $\eta_1=\pm1$. You should be familiar with double-well curves of free energy of this sort from your studies of thermodynamics and phase diagrams. In those cases, the x-axis will often represent composition of a phase and the two wells might have different depths. Phase field models can be applied to problems of exactly this sort, but they require a form of the evolution equations in which the average composition of the sample is conserved. As I pointed out above, this makes the mathematics a little more complicated and I want to keep things as simple as possible for this introduction. So in our case, the x-axis $\eta_1$ simply represents the extent to which the material is in phase 1. When we interpret the phase of the system at a given $r$ we ignore the sign of the phase variables and so $\eta_1(r)=1$ and $\eta_1(r)=-1$ are both interpreted as meaning that the system is in phase 1 at point $r$.

### Adding a second phase
So far, our system is unlikely to do anything exciting. We have only defined one phase and, given the shape of the free energy curve above, at each point in space the value of $\eta_1$ will tend to $+1$ or $-1$, both of which indicate that the system is in phase 1. To obtain interesting behaviour we need to add in a second phase variable. As a first attempt, let's try writing the free energy of this two phase system as:

$$
f_0(r) = 
-\frac{\alpha}{2}\left( \left(\eta_1(r)\right)^2 + \left(\eta_2(r)\right)^2 \right)
+\frac{\beta}{4} \left( \left(\eta_1(r)\right)^4 + \left(\eta_2(r)\right)^4 \right)
.
$$

We can visualise this new $f_0$ as a heat-map, in which dark areas indicate minima in the function:

In [ ]:
etamin = -1.5 # Set the limits of the plot
etamax = 1.5
nvals = 100
alpha = 1.0
beta = 1.0
f = np.zeros([100,100]) # Array to hold values of free energy f at different values of eta1 and eta2  
# The following loops iterate through pairs of values of eta1 and eta2 calculating the free energy at each value
for i in range(nvals):
    eta1 = etamin + i*(etamax-etamin)/nvals
    for j in range(nvals):
        eta2 = etamin + j*(etamax-etamin)/nvals
        f[i,j] = - alpha/2*eta1**2 + beta/4*eta1**4 - alpha/2*eta2**2 + beta/4*eta2**4
# Now plot a heatmap of the value of f as a function of eta1 and eta2
im=plt.imshow(f,cmap = cm.Greys_r, extent=[etamin,etamax,etamin,etamax], vmin = np.min(f), vmax=0.1)
plt.xlabel('$\eta_1$')
plt.ylabel('$\eta_2$')
plt.show()

The minima in free energy are shown by the dark areas. This isn't quite what we want. There are regions of low energy at $\eta_1=\pm1$ and $\eta_2=\pm1$, but the deepest minima are at the corners of the plot, at $(\eta_1,\eta_2)=(\pm1,\pm1)$ and $(\eta_1,\eta_2)=(\pm1,\mp1)$, which implies that both phases are stable simultaneously. For example, the system might tend towards the state represented by the well (dark spot) in the top right in which $\eta_1=1$, indicating that we are in phase 1, *and* $\eta_2=1$, indicating that we are *also* in phase 2! This form for $f_0(r)$ clearly will not represent physically reasonable behavior. It is the job of an additional, final term in $f_0$

$$
\gamma\sum_{i=1}^p\sum_{j\neq i,j=1}^p \left(\eta_i(r)\right)^2\left(\eta_j(r)\right)^2
$$

to exclude this possiblity. If we add this in to the two-phase case, we have:

$$
f_0(r) = 
-\frac{\alpha}{2}\left( \left(\eta_1(r)\right)^2 + \left(\eta_2(r)\right)^2 \right)
+\frac{\beta}{4} \left( \left(\eta_1(r)\right)^4 + \left(\eta_2(r)\right)^4 \right)
+ \gamma\left( \left(\eta_1(r)\right)^2\left(\eta_2(r)\right)^2 + \left(\eta_2(r)\right)^2\left(\eta_1(r)\right)^2 \right),
$$
$$
f_0(r) = 
-\frac{\alpha}{2}\left( {\eta_1}^2 + {\eta_2}^2 \right)
+\frac{\beta}{4} \left( {\eta_1}^4 + {\eta_2}^4 \right)
+ \gamma\left( {\eta_1}^2{\eta_2}^2 + {\eta_2}^2{\eta_1}^2 \right)
,
$$
where in the second line I have left the dependence of the phase variables on $r$ aas implicit to make the form clearer. The below code plots this full form of $f_0$ for the two-phase case:

In [ ]:
etamin = -1.5
etamax = 1.5
nvals = 100
alpha = 1.0
beta = 1.0
gamma = 1.0
f = np.zeros([100,100])       
for i in range(nvals):
    eta1 = etamin + i*(etamax-etamin)/nvals
    for j in range(nvals):
        eta2 = etamin + j*(etamax-etamin)/nvals
        f[i,j] = - alpha/2*eta1**2 + beta/4*eta1**4 - alpha/2*eta2**2 + beta/4*eta2**4
        f[i,j] = f[i,j] + gamma*(eta1**2*eta2**2 + eta2**2*eta1**2)
im=plt.imshow(f,cmap = cm.Greys_r, extent=[etamin,etamax,etamin,etamax], vmin = np.min(f), vmax=0.1)
plt.xlabel('$\eta_1$')
plt.ylabel('$\eta_2$')
plt.show()

This fixes things and means that only one phase at a time can be stable at a given point in space at a given time: in the minima where $|\eta_1|=1$ we have $\eta_2=0$ and vice versa.

#### <span style="color: red"> Task 2:</span> Explore the effect of varying the value of  `gamma`
Try replotting the above heatmap for values of `gamma` from 0.1 to 2.0. How does the free energy function change? What happens at `gamma=0.25`? What happens at smaller values of `gamma`, e.g. `gamma=0.1` 

### The total free energy of the system

At the start of the notebook, we gave the expression for the free energy of the system:

$$
F = \int \mathrm{d}^3r \left[
f_0(\eta_1(r),\eta_2(r),\dots,\eta_p(r))
+ \sum_i \frac{\kappa}{2}(\nabla \eta_i(r))^2
\right].
$$


We have now spent a little time considering the form of the expression for the bulk free energy $f_0$, which means this expression becomes:


$$
F = \int \mathrm{d}^3r \left[
-\frac{\alpha}{2} \sum_{i=1}^p \left(\eta_i(r)\right)^2
+\frac{\beta}{4} \sum_{i=1}^p \left(\eta_i(r)\right)^4
+\gamma\sum_{i=1}^p\sum_{j\neq i,j=1}^p \left(\eta_i(r)\right)^2\left(\eta_j(r)\right)^2
+ \sum_{i=1}^p \frac{\kappa}{2}(\nabla \eta_i(r))^2
\right].
$$


Each point in space will have associated with it values of the phase variables $\eta_1, \eta_2,\dots,\eta_p$ which indicate the phase at that point. The form of the expression that we gave above, including the $\gamma$ term, ensures that only one of these phases should be present at each point (i.e. that only one of the phase variables will have the value $\pm 1$ and all others will be zero). To calculate the total free energy of the system we integrate over all points in space (essentially we add up the energy contributions from every point in space). This is what is shown above.

#### Mathematical aside
You will see that the calculation of $F$ involves integrating an integrand that involves functions $\eta_1(r)$ and so on, that are themselves functions of the position in space $r$. If we change the shape of the $\eta$-functions then the value of $F$ will, in general, also change. This means that $F$ is something called a *functional*: a function of a function (or in this case a function of several functions). Functionals occur in many fields of science and are a useful concept, though the mathematics of their use is beyond the scope of this course. Look them up if you would like to find out more.



### The interfacial energy
We have not yet explored the effect on the free energy of the term involving $\kappa$. To help us do this, we will assume a particular form for the $\eta$-functions. Remember that these $\eta$-functions describe the state of the system (the distribution of phases in space) and so in a real simulation the form of these functions will be determined by the evolution of the system, but we have not yet discussed this. So for now, we will assume that the functions take a model form, the same as the one we examined right at the start of the notebook. To make things particularly simple, we will consider once again a system with a single phase:

$$
F = \int \mathrm{d}^3r \left[
-\frac{\alpha}{2} \left(\eta_1(r)\right)^2
+\frac{\beta}{4}  \left(\eta_1(r)\right)^4
+ \frac{\kappa}{2}\left(\frac{\mathrm{d} \eta_1(r)}{\mathrm{d} r}\right)^2
\right].
$$

We will further assume that the form taken by $\eta_1(r)$ is:

$$
\eta_1(r) = \frac{2}{1+\exp(r/\sigma)}-1,
$$

in which the value of $\sigma$ controls the interfacial width.

This is the form that we plotted in the very first plot in this notebook, although I have amended it a little to give a variation in $\eta_1$ from -1 to 1. The form of $F$ above requires the derivative of $\eta$ and we can calculate the derivative of our model form for $\eta$ analytically:

$$
\frac{\mathrm{d} \eta_1(r)}{\mathrm{d} r} = \frac{-2\exp(r/\sigma)}{\sigma\left(1+\exp (r/\sigma)\right)^2}
$$

Now we need some Python code to calculate the free energy $F$ associated with this form for $\eta(r)$.


Take a look at the code below. It defines the function eta and its derivative in the form discussed above. It also defines another function `F()`, which returns the value of the free energy $F$ by summing the contributions from each point $r$ in the system. The rest of the code simply plots some useful functions and prints the value of $F$.

In [ ]:
# Define a model form for eta
def eta(r,sigma):
    return 2.0/(1.0+np.exp(r/sigma))-1.0

# Derivative of eta
def deta(r,sigma):
    return -2.0*np.exp(r/sigma)/sigma/(1.0+np.exp(r/sigma))**2

# Function to calculate free energy of whole system
def F(x,alpha,beta,kappa,sigma):
    F_val = 0.0
    for a in x:
        # Iterate ove all points in the system, calculating the local free energy and adding it to the total
        F_val = F_val - alpha/2.0*eta(a,sigma)**2 + beta/4.0*eta(a,sigma)**4 + kappa/2.0*deta(a,sigma)**2
    return F_val

# Now plot the results
alpha = 1.0
beta = 1.0
kappa = 1.0
sigma = 1.0
r=np.linspace(-5,5,100)

fig, ax1 = plt.subplots()
ax1.plot(r,eta(r,sigma),'r-', label="$\eta_1$")
ax1.legend()
ax2 = ax1.twinx()
ax2.plot(r,-alpha/2.0*eta(r,sigma)**2 + beta/4.0*eta(r,sigma)**4,'g-', label="Bulk energy")
ax2.plot(r,kappa/2.0*deta(r,sigma)**2,'k-', label="Interfacial energy")
ax2.legend()
ax1.set_xlabel('Position, $r$')
ax1.set_ylabel('Phase variable ')
ax2.set_ylabel('Energy')

print('Total free energy = ' + str(F(r,alpha,beta,kappa,sigma)))  

You will see that at low and high $r$, where the phase variable $\eta_1$ adopts its stable values of 1.0 and -1.0, the contribution from the bulk energy term is negative. The central region where the value of $\eta_1$ is varying (the interfacial region) gives a positive contribution to the free energy via the interfacial energy term (and also a positive contribution from the bulk energy term since $\eta_1$ is not at either of its stable values in this region).

#### <span style="color: red"> Task 3:</span> Find the optimal interfacial width
In a real phase field simulation the shape of $\eta_1$ would be determined by the evolution of the system towards its lowest energy state, i.e. $\eta_1(r)$ would take whatever shape in space was required to give the lowest total energy $F$ for the whole system. We haven't yet talked about how to evolve our system, but we can mimic this process by varying the shape of $\eta_1$ ourselves. Remember that the shape of $\eta_1$ is controlled by the value of the parameter $\sigma$.

Think about how the total free energy varies in the simplified example above as we increase the width of the interface region (by increasing `sigma`). The contribution from bulk free energy increases (a smaller negative number). This is because less of the material is in the stable phase ($\eta_1=\pm1$ over a smaller range of $r$). However, the interfacial term becomes smaller (a smaller positive number) because the steepnes with which $\eta_1$ varies in space is reduced (and the interfacial energy depends on the square of the gradient of $\eta_1$). At some value of `sigma` the total free energy of this system will be a minimum.

Find the value of `sigma` (and hence the shape of $\eta_1$) that minimises $F$. You could use the code above to search by hand (i.e. try different values of `sigma` and see how `F` varies) or you could use python to plot a graph of $F$ against $\sigma$ (note that I have deliberately set up the python function `F()` so that `sigma` is a parameter to help with this - you would need to define a loop over values of `sigma` and record the corresponding values of `F` ready to plot).

#### <span style="color: red"> Task 4:</span> How does the value of  `kappa` ($\kappa$) affect the optimal interface width
The interfacial energy term also involves the parameter $\kappa$. Change the value of `kappa` to 2.0 and see what happens to the optimal interface width.

### Choice of interfacial width
You should have found from the above tasks that the value of $\kappa$ in the functional for the free energy $F$ controls the optimal interfacial width. Remember that the phase field approach models diffuse intefaces primarily as a way to make the simulations more efficient. Many of the interfaces in real systems are quite sharp (consider, for example, the interface between a precipitate phase and the matrix phase in a metallic alloy). Does this suggest that we should make the value of $\kappa$ very small so that the interfaces in our simulation are also very sharp? What would be the consequences of doing this for the computational power required for our simulations? We will return to this question in a short while.

### The evolution of the system
The phase field method is widely used to study the evolution of different types of microstructure. Since the state of the system is defined by the phase variables, we therefore need an equation that determines the evolution of these variables. The equation has the form:

$$
\frac{\partial \eta_i}{\partial t} = - L\frac{\delta F}{\delta \eta_i}.
$$

To fully understand what is going on with this equation requires mathematics that is beyond the scope of the course (the term on the right is something called a *functional derivative* if you are interested to find out more on your own). However, the equation expresses an intuitive idea: each phase variable will evolve in time in such a way that the free energy $F$ gets smaller. The functional derivative ${\delta F}/{\delta \eta_i}$ gives the rate of change of $F$ as the distribution of the phase variable $\eta_i$ is changed and the minus sign drives the evolution downhill in $F$. The constant $L$ then controls the rate at which the evolution takes place and so it represents the *mobility* of the interfaces between phases. Since we have an expression for the free energy of the system $F$ we can substitute this into the above equation to arrive at an expression for the evolution involving more familiar notation (ordinary derivatives rather than functional derivatives):

$$
\frac{\textrm{d}\eta_i(r,t)}{\textrm{d}t} = - L \left\{ -\alpha\, \eta_i(r,t) + \beta \left(\eta_i(r,t)\right)^3 + 2\gamma \eta_i(r,t) \sum_{j\neq i} \left(\eta_j(r,t)\right)^2\right\} + L \kappa\nabla^2\eta_i(r,t).
$$

Note: to get this result, we have carried out the functional derivative on the expression for $F$. We won't go through this, but if you try making the substitution yourself and simply take a normal derivative, treating the functions $\eta_i$ as if they were just ordinary scalar quantities, then you will get the answer above (whilst taking functional derivatives is in general more complicated than taking ordinary derivatives, it is often the case that following the recipes for ordinary differentiation works out fine).

## Using Python to implement a phase field model
We have now gone back over the theory that we covered in the lecture and explored some of the properties of the free energy model using our python notebook. In this section we will develop some code to implement a phase field model within the notebook. We will go through this step-by-step as slowly as we can, but this is a complicated problem to solve and we do not expect all of you to follow all of what is going on. However, you should try your best to get as deep an understanding as you can of what is going on.

### Representing the phase variables

So far we have been representing the phase field model via equations. These make use of continous functions. So, for example, our function $\eta_1(r)$ has a value for every conceivable value of $r$. However, in a computer we deal with discrete data, and so we need to find a way to implement the phase field model under this constraint. Of course, the phase field model is just a set of differential equations and you have already spent some time looking at how to deal with these in a computer in the first half of the module. 

The first thing we need to do is choose a way to represent the values of our phase variables $\eta_i(r)$  as a function of position in space. We do this by defining a suitable array. Of course, we will need to discretise space on a grid and so we will need to choose a spatial length scale for this discretisation:

In [ ]:
n = np.array([10,10]) # Size of grid in x and y directions
dr = 0.5 # grid scale in distance units
p = 2 # number of phase variables
eta = np.zeros([n[0],n[1],p]) # Array to hold values of phase variables

Above we are setting a system with two phase variables on a 10 by 10 grid in which each square is 0.5 by 0.5 distance units.

### Interfacial width, again

#### <span style="color: red"> Task 5:</span> Exploring the interfacial width on a discrete grid.
The code below plots the value of a phase variable for a given `sigma` (we are using a model function for this, remember that in a real simulation the phase variables have a form that is the result of minimising the total free energy). The value of the function is shown using a discrete grid with a spacing controlled by `dr`.

In the given example, the variation in `eta` is captured by only around 7 points. Is this enough? Try changing `dr` to 0.5. This should give a smoother representation of `eta` but now we need twice as many points. Try changing `sigma` to 0.1 and find a value of `dr` that gives a reasonably smooth representation of `eta`.

In [ ]:
def eta(r,sigma):
    return 2.0/(1.0+np.exp(r/sigma))-1.0

rmax = 5.0
rmin = -5.0
dr = 1.0
sigma = 0.5
r=np.linspace(-5,5,int((rmax-rmin)/dr)+1)

plt.plot(r,eta(r,sigma),'ro-', label="$\eta_1$")
plt.xlabel('Position, $r$')
plt.ylabel('Phase variable ')

You should now see why we often need to consider interfaces in the phase field model that are more diffuse than the ones that they represent in the real world. If we make the interfaces in our model very sharp, then we need to use a very fine grid to give a smooth representation of the phase variables. This then means that to represent a given volume of material we will need many more grid points and the computational cost of our model will increase dramatically. The width of the interfaces (controlled by $\kappa$) is therefore a compromise between a faithful representation of reality and a computationally tractable model.

### Discretising the evolution of the system
Just as we must use an array to hold the values of the phase variables in space, we also need to work with a discrete system in time. This means that we need to use a discrete version of the equation for the evolution of the system:

$$
\frac{\partial \eta_i}{\partial t} = - L \frac{\delta F}{\delta \eta_i}.
$$

In this case we can use a very simple approach called the *Euler method*. We choose a discrete timestep $\Delta t$ and then evolve the system repeatedly through a series of timesteps according to the finite difference equation:

$$
\eta_i(r,t+\Delta t) = \eta_i(r,t) + \frac{\partial \eta_i(r,t)}{\partial t}\Delta t.
$$

At a time $t+\Delta t$, $\eta$ will take the value it did at time $t$ plus a small amount determined by the rate of change of $\eta$ with time, as given by ${\partial \eta_i(r,t)}/{\partial t}$.

We will implement this evolution shortly.

### The `PhaseField()` class: object-oriented programming
One of the biggest strengths of using computers to solve materials science problems is the ability to reuse the tools that we build. We have seen this a lot so far in our use of functions, loops and so on. A really useful concept in many modern programming languages is the idea of objects and object-oriented programming. This subject can get pretty complicated and we do not need to get into the details (you can find out more in your own time if you are interested). The basic idea is to structure code in such a way that if we are dealing with a certain kind of object, then all of the functions and variables associated with that object are gathered together. This is best illustrated with an example and it makes sense to work on the problem we are examining today....

The code below defines a *class* called `PhaseField()`. Think of this as a container for everything that we need to know about a phase field model and everything we need to do to it.

In [ ]:
class PhaseField():
    
    def __init__(self,n,p,dx,alpha,beta,gamma,kappa,L):
        # Model size
        self.n = np.array(n)
        self.dx = dx
        self.dx2 = self.dx*self.dx
        self.p = p
        # Model parameters
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.kappa = np.full(p,kappa)
        self.L = L
        # Initialise phase variables (zero at all points initially)
        self.eta = np.zeros([self.n[0],self.n[1],self.p])

Functions associated with the object contained in a class are called *member functions*. This phase field model has one member function called `__init__()`. This is a special function and so its name has a special form. All classes need such a function and it does the important job of setting up the object. We'll define some more normal looking functions later. The `__init__()` method also defines *member variables*, such as `self.alpha`, which store key properties or data associated with our object. The parameters passed to the function are stored in these variables.

So far, the class simply describes a phase field object in an abstract way. If we want to generate a specific example of such an object then we need to create an *instance* of the object. This is done as follows:

In [ ]:
mymodel = PhaseField(n=[200,200],p=20,dx=0.5,alpha=1.0,beta=1.0,gamma=1.0,kappa=1.0,L=1.0)

The variable `mymodel` now contains a phase field model. We call the name of the class, `PhaseField()`, as if it were a function, to create a new instance of the class. This syntax actually calls the `__init()__` member function and creates all the variables associated with our particular phase field model, such as the three-dimensional array `eta[]` containing the value of the phase variables and the constants `alpha`, `beta` and so on. Note that there is no particular need to give the parameter names when calling `PhaseField()`. We have done this just to aid readability, since there are so many parameters. We could just have written:

`PhaseField([200,200], 20, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0)`

We can examine the values of these parameters for our model (which is to say we can access the contents of the member variables) using the "dot" syntax as follows:

In [ ]:
print(mymodel.alpha)

In [ ]:
print(mymodel.eta[0,0,0])

### Initialising and visualising the model
The code we have for our class so far will create a basic phase field model, but all it contains is a description of the parameters and the phase variables. It doesn't actually do anything! The code below adds some more member functions. Note that we repeat the whole class definition, including the `__init__()` function. 

The new code adds two ways to initialise the values of the phase variables stored in the array `eta[]`. One of these leaves most of the values at zero, but sets some random cells to 1 to function as seeds for evolution. The other attempts to mimic a liquid state by setting all values in the array to a random value with a fixed scale of fluctuation.

It also includes three functions for outputting the model. Two of these plot the array `eta[]` to the notebook output cell, either in greyscale or colour. The third produces a .png image file.

Run the code below to update the class definition.

In [ ]:
class PhaseField():
    
    def __init__(self,n,p,dx,alpha,beta,gamma,kappa,L):
        # Model size
        self.n = np.array(n)
        self.dx = dx
        self.dx2 = self.dx*self.dx
        self.p = p
        # Model parameters
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.kappa = np.full(p,kappa)
        self.L = L
        # Initialise phase variables
        self.eta = np.zeros([self.n[0],self.n[1],self.p])
        
    def initializeseedsrandom(self,ngrains):
        # Function to fill the phase array with random numbers at ngrains points (mimic a set of crystal seed nuclei)
        for s in range(ngrains):
            i = np.random.randint(self.n[0])
            j = np.random.randint(self.n[1])
            p = np.random.randint(self.p)
            self.eta[i,j,p] = 1
        
    def initializeliquid(self,fluctuation):
        # Function to fill the phase array with random numbers at all points (mimic a liquid state)
        self.eta = fluctuation*2.0*(np.random.random_sample((self.n[0],self.n[1],self.p)) - 0.5)

    # The following three functions just provide different ways of visualising the state of the system
    # Don't worry too much about the details now. Come back to them later if you are interested
    def plotphasesgray(self):
        phasemap = np.zeros([self.n[0],self.n[1]])       
        for s in range(self.p):
            phasemap[:,:] = phasemap[:,:]+self.eta[:,:,s]**2
        im=plt.imshow(phasemap,cmap = cm.Greys_r)
        im.axes.get_xaxis().set_visible(False)
        im.axes.get_yaxis().set_visible(False)
        plt.show()
    
    def plotphasescolor(self):
        hsvmap = np.zeros([self.n[0],self.n[1],3]) 
        for s in range(self.p):
            hsvmap[:,:,0] = hsvmap[:,:,0] + (self.eta[:,:,s] > 0.5*self.eta.max()).astype(float)*(self.eta[:,:,s] == self.eta.max(axis=2)).astype(float)*(s+1)/self.p
            hsvmap[:,:,1] = 1.0 
            hsvmap[:,:,2] = hsvmap[:,:,2] + self.eta[:,:,s]**2
        hsvmap[:,:,2] = (hsvmap[:,:,2] - np.min(hsvmap[:,:,2]))/(np.max(hsvmap[:,:,2])-np.min(hsvmap[:,:,2]))
        im=plt.imshow(hsv_to_rgb(hsvmap))
        im.axes.get_xaxis().set_visible(False)
        im.axes.get_yaxis().set_visible(False)
        plt.show()
    
    def savephasesgray(self,folder,filename):
        my_dpi =int(0.5*np.sum(self.n)/5.0)
        phasemap = np.zeros([self.n[0],self.n[1]])       
        for s in range(self.p):
            phasemap[:,:] = phasemap[:,:]+self.eta[:,:,s]**2
        img.imsave('./'+folder+'/'+filename+'.png', phasemap)

#### <span style="color: red"> Task 6:</span> Try out the initialisation and visualisation functions
Play with the code below to create a phase field model, intialise it and visualise it. Try varying the number of cells in the grid, the type of intialisation and the method for visualisation (note that to write an image to a file you will need to make sure that the specified folder already exists).

In [ ]:
# Create a model
mymodel = PhaseField(n=[50,50],p=20,dx=0.5,alpha=1.0,beta=1.0,gamma=1.0,kappa=1.0,L=1.0)

# Initialise the model (selectively comment and uncomment the lines to try different methods of initialisation)
#mymodel.initializeseedsrandom(10)
mymodel.initializeliquid(0.001)

#Output the model (selectively comment and uncomment the lines to try different forms of output)
mymodel.plotphasescolor()
#mymodel.plotphasesgray()
#mymodel.savephasesgray('tempfolder','tempfile')


### Completing the class - time evolution
We almost have everything we need to run a phase field simulation. We will now repeat the class definition one more time and add in the functions required to evolve the phase variables. The most important function below is the one called `evolve()`. This encodes all of the physics of the model that we discused above in a discretised form. To work out the change in the phase variable in a given cell over a timestep $\Delta t$, we need to know the values of the phase in the neighbouring cells. The final function `getneighbours()` is just a little helper function to return a list of the neighours. We don't expect you all to fully understand the contents of the `evolve()` function, but you may recognise some features of the discrete representation of the phase field equations from your session on finite difference methods. 

(Note that the implementation below might look a little bit different from how the equations were written above. This is to improve the efficiency of the code.)

Run the code below to update the class definition and then we will use the remainder of the session to try out the model.

In [ ]:
class PhaseField():
    
    def __init__(self,n,p,dx,alpha,beta,gamma,kappa,L):
        # Model size
        self.n = np.array(n)
        self.dx = dx
        self.dx2 = self.dx*self.dx
        self.p = p
        # Model parameters
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.kappa = np.full(p,kappa)
        self.L = L
        # Initialise phase variables
        self.eta = np.zeros([self.n[0],self.n[1],self.p])
        
    def initializeseedsrandom(self,ngrains):
        for s in range(ngrains):
            i = np.random.randint(self.n[0])
            j = np.random.randint(self.n[1])
            p = np.random.randint(self.p)
            self.eta[i,j,p] = 1
        
    def initializeliquid(self,fluctuation):
        self.eta = fluctuation*2.0*(np.random.random_sample((self.n[0],self.n[1],self.p)) - 0.5)
        
    def getneighbours(self,i,j):
        nn = np.array([
                [ [(i-1)%self.n[0],j], [(i+1)%self.n[0],j], [i,(j-1)%self.n[1]], [i,(j+1)%self.n[1]]],
                [ [(i-2)%self.n[0],j], [(i+2)%self.n[0],j], [i,(j-2)%self.n[1]], [i,(j+2)%self.n[1]]]
            ])
        return nn
    
    def evolve(self,dt):
        for i in range(self.n[0]):
            for j in range(self.n[1]):
                nn = self.getneighbours(i,j)
                # Calculate second derivative of eta in space
                grad2i = (
                    0.5 * np.sum(self.eta[nn[0,:,0],nn[0,:,1],:], axis=0)
                    + 0.25 * np.sum(self.eta[nn[1,:,0],nn[1,:,1],:], axis=0)
                    - 3.0 * self.eta[i,j,:]
                ) / self.dx2
                # Calculate the sum of eta^2 
                sumeta2 = np.sum(self.eta*self.eta,axis=2)
                # Calculate the rate of change of eta
                deta =  -self.L * (
                    - self.alpha*self.eta[i,j,:]
                    + self.beta*self.eta[i,j,:]**3.0
                    + 2.0*self.gamma*self.eta[i,j,:]*(sumeta2[i,j]-self.eta[i,j,:]**2.0)
                    - self.kappa*grad2i
                )
                # Calculate the new value of eta
                self.eta[i,j,:] = self.eta[i,j,:]+deta*dt

    def plotphasesgray(self):
        phasemap = np.zeros([self.n[0],self.n[1]])       
        for s in range(self.p):
            phasemap[:,:] = phasemap[:,:]+self.eta[:,:,s]**2
        im=plt.imshow(phasemap,cmap = cm.Greys_r)
        im.axes.get_xaxis().set_visible(False)
        im.axes.get_yaxis().set_visible(False)
        plt.show()
    
    def plotphasescolor(self):
        hsvmap = np.zeros([self.n[0],self.n[1],3]) 
        for s in range(self.p):
            hsvmap[:,:,0] = hsvmap[:,:,0] + (self.eta[:,:,s] > 0.5*self.eta.max()).astype(float)*(self.eta[:,:,s] == self.eta.max(axis=2)).astype(float)*(s+1)/self.p
            hsvmap[:,:,1] = 1.0 
            hsvmap[:,:,2] = hsvmap[:,:,2] + self.eta[:,:,s]**2
        hsvmap[:,:,2] = (hsvmap[:,:,2] - np.min(hsvmap[:,:,2]))/(np.max(hsvmap[:,:,2])-np.min(hsvmap[:,:,2]))
        im=plt.imshow(hsv_to_rgb(hsvmap))
        im.axes.get_xaxis().set_visible(False)
        im.axes.get_yaxis().set_visible(False)
        plt.show()
    
    def savephasesgray(self,folder,filename):
        my_dpi =int(0.5*np.sum(self.n)/5.0)
        phasemap = np.zeros([self.n[0],self.n[1]])       
        for s in range(self.p):
            phasemap[:,:] = phasemap[:,:]+self.eta[:,:,s]**2
        img.imsave('./'+folder+'/'+filename+'.png', phasemap)

### Running a simulation
The code below runs a phase field simulation which could represent solidification from a melt and the subsequent evolution of the grain structure. In this case, each phase (below we have 20 possible phases) can be taken to represent a given orientation of the crystal structure. The regions of different "phase" are then individual grains and the interfaces between them are grain boundaries.

This simulation begins with the phase variables set to random variables (to represent the molten state). It runs for 200 time steps and outputs an image of the model every 50 steps. This simulation runs for just long enough for a clear pattern of grains to emerge.

In [ ]:
mymodel = PhaseField(n=[40,40],p=20,dx=0.5,alpha=1.0,beta=1.0,gamma=1.0,kappa=0.1,L=1.0)
mymodel.initializeliquid(0.001)
nsteps = 250
outperiod = 50
for i in range(nsteps):
    mymodel.evolve(0.1)
    if i%outperiod==0:
        mymodel.plotphasesgray()

#### <span style="color: red"> Task 7:</span> Repeat the above simulation using a value for `kappa` of 0.3
Copy the code from above into a new cell and run again with the new value of `kappa`. What do you expect this larger value of kappa to do? Does the simulation agree with your expectations?

#### <span style="color: red"> Task 8:</span> Run a simulation of grain coarsening
Copy the simulation code into a new cell again, with the original value of `kappa=0.1`. Change `nsteps` to 1000 to run a longer simulation. You should also change `outperiod` to a larger value, say 100, to avoid plotting too many images. Run the code and examine the sequence of images. What behaviour do you observe? How does this conform to your expectations based on what you know about how grain structures evolve in real materials?

## That's it!
We have explored the theory behind the phase field model and implemented a version of it through a python class. You can now play around with this, if you wish. Try the different ways of visualising the evolving state of the model. Perhaps add a function to return the energy so that you can plot a graph of the evolution of the free energy of the simulation over the course of a simulation. What happens if you vary the value of `gamma`? (Remember that we explored how this affected $f_0$ earlier in the notebook.) Can you think of ways to make the model more realistic: for example, how might you add anisotropy to the interfacial energies? Perhaps you could plot the value of the different phase variables in one dimension along a cross-section through the grain structure.